# **DOWNLOADING KAGGLE JSON FILE**

In [ ]:
!gdown 1UjlEITId68glUxwRIz7VhXkuW2vcLma2

Downloading...
From: https://drive.google.com/uc?id=1UjlEITId68glUxwRIz7VhXkuW2vcLma2
To: /content/kaggle.json
100% 66.0/66.0 [00:00<00:00, 296kB/s]


# **DOWNLOADING DATASET**

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!mv kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d joe1995/div2k-dataset

Dataset URL: https://www.kaggle.com/datasets/joe1995/div2k-dataset
License(s): unknown
100% 3.70G/3.71G [00:19<00:00, 276MB/s]
100% 3.71G/3.71G [00:19<00:00, 206MB/s]


In [ ]:
!unzip div2k-dataset -d /content/data

Archive:  div2k-dataset.zip
  inflating: /content/data/DIV2K_train_HR/DIV2K_train_HR/0001.png  
  inflating: /content/data/DIV2K_train_HR/DIV2K_train_HR/0002.png  
  inflating: /content/data/DIV2K_train_HR/DIV2K_train_HR/0003.png  
  inflating: /content/data/DIV2K_train_HR/DIV2K_train_HR/0004.png  
  inflating: /content/data/DIV2K_train_HR/DIV2K_train_HR/0005.png  
  inflating: /content/data/DIV2K_train_HR/DIV2K_train_HR/0006.png  
  inflating: /content/data/DIV2K_train_HR/DIV2K_train_HR/0007.png  
  inflating: /content/data/DIV2K_train_HR/DIV2K_train_HR/0008.png  
  inflating: /content/data/DIV2K_train_HR/DIV2K_train_HR/0009.png  
  inflating: /content/data/DIV2K_train_HR/DIV2K_train_HR/0010.png  
  inflating: /content/data/DIV2K_train_HR/DIV2K_train_HR/0011.png  
  inflating: /content/data/DIV2K_train_HR/DIV2K_train_HR/0012.png  
  inflating: /content/data/DIV2K_train_HR/DIV2K_train_HR/0013.png  
  inflating: /content/data/DIV2K_train_HR/DIV2K_train_HR/0014.png  
  inflating: /conten

# **Downloading Necessary Libraries**

In [ ]:
!pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 37.1 MB/s eta 0:00:00


# **IIG MODULE DEFINITION**

In [ ]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from PIL import Image
import pywt
import numpy as np
import cv2

class DWTForward(nn.Module):
    def __init__(self, wave='haar'):
        super(DWTForward, self).__init__()
        self.wave = wave

    def forward(self, x):

        pad_size = (x.shape[2] % 2, x.shape[3] % 2)
        if pad_size[0] != 0 or pad_size[1] != 0:
            x = F.pad(x, (0, pad_size[1], 0, pad_size[0]), mode='reflect')

        coeffs = pywt.dwt2(x.cpu().detach().numpy(), self.wave)
        x_LL, (x_LH, x_HL, x_HH) = coeffs

        x_LL = torch.tensor(x_LL).to(x.device)
        x_LH = torch.tensor(x_LH).to(x.device)
        x_HL = torch.tensor(x_HL).to(x.device)
        x_HH = torch.tensor(x_HH).to(x.device)

        return x_LL, x_LH, x_HL, x_HH

class DWTInverse(nn.Module):
    def __init__(self, wave='haar'):
        super(DWTInverse, self).__init__()
        self.wave = wave

    def forward(self, x_LL, x_LH, x_HL, x_HH):

        coeffs = (x_LL.cpu().detach().numpy(), (x_LH.cpu().detach().numpy(), x_HL.cpu().detach().numpy(), x_HH.cpu().detach().numpy()))
        x = pywt.idwt2(coeffs, self.wave)


        x = torch.tensor(x).to(x_LL.device)
        return x

class InvertibleBlock(nn.Module):
    def __init__(self, in_channels):
        super(InvertibleBlock, self).__init__()
        self.phi = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, padding_mode='reflect'),
            nn.ReLU(),
            nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, padding_mode='reflect')
        )
        self.rho = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, padding_mode='reflect'),
            nn.ReLU(),
            nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, padding_mode='reflect')
        )
        self.eta = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, padding_mode='reflect'),
            nn.ReLU(),
            nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, padding_mode='reflect')
        )
        self.alpha = nn.Sigmoid()

    def forward(self, x_L, x_H):
        x_L_next = x_L + self.phi(x_H)
        x_H_next = x_H * torch.exp(self.alpha(self.rho(x_L_next))) + self.eta(x_L_next)
        return x_L_next, x_H_next

    def inverse(self, x_L_next, x_H_next):
        x_H = (x_H_next - self.eta(x_L_next)) / torch.exp(self.alpha(self.rho(x_L_next)))
        x_L = x_L_next - self.phi(x_H)
        return x_L, x_H
# Separate Loss Functions Implementation
class HighPassEstimator(nn.Module):
    def __init__(self, in_channels):
        super(HighPassEstimator, self).__init__()
        self.residual_block = nn.Sequential(
            nn.Conv2d(in_channels * 2, in_channels * 3, kernel_size=3, padding=1,padding_mode='reflect'),
            nn.ReLU(),
            nn.Conv2d(in_channels *3, in_channels * 3, kernel_size=3, padding=1,padding_mode='reflect'),
            nn.ReLU(),
            nn.Conv2d(in_channels *3, in_channels * 2, kernel_size=3, padding=1,padding_mode='reflect'),
            nn.ReLU(),
            nn.Conv2d(in_channels *2, in_channels, kernel_size=3, padding=1,padding_mode='reflect'),
            nn.ReLU(),

        )

    def forward(self, x_L, x_H):
        x = torch.cat((x_L, x_H), dim=1)
        return self.residual_block(x)

class HighPassFusion(nn.Module):
    def __init__(self):
        super(HighPassFusion, self).__init__()

        self.conv_fusion = nn.Sequential(
            nn.Conv2d(9, 3, kernel_size=3, padding=1, padding_mode='reflect'),
            nn.ReLU()
        )

    def forward(self, x_H_concat):
        x_H_fused = self.conv_fusion(x_H_concat)
        return x_H_fused


class IIRModule(nn.Module):
    def __init__(self, in_channels, num_blocks=4):
        super(IIRModule, self).__init__()
        self.dwt = DWTForward()
        self.iwt = DWTInverse()
        self.invertible_blocks = nn.ModuleList([InvertibleBlock(in_channels) for _ in range(num_blocks)])
        self.highpass_estimator = HighPassEstimator(in_channels)

    def forward(self, x):
        x_L, x_LH, x_HL, x_HH = self.dwt(x)

        x_H_concat = torch.cat([x_LH, x_HL, x_HH], dim=1)
        high_pass_fusion = HighPassFusion()
        x_H = high_pass_fusion(x_H_concat)
        for block in self.invertible_blocks:
            x_L, x_H = block(x_L, x_H)
        return x_L

    def sample_z(self, x_L):
        z_sample = torch.randn_like(x_L)
        return z_sample

    def inverse(self, x_L, tensor_r=None, use_saved=False):
        if tensor_r is None:
            tensor_r = [torch.zeros_like(x_L) for _ in range(len(self.invertible_blocks))]

        x_H = tensor_r[-1]

        for i in range(len(self.invertible_blocks) - 1, -1, -1):
            x_L, x_H = self.invertible_blocks[i].inverse(x_L, x_H)
        x_LH = self.highpass_estimator(x_L, x_H)
        x_HL = self.highpass_estimator(x_L, x_H)
        x_HH = self.highpass_estimator(x_L, x_H)
        x_reconstructed = self.iwt(x_L, x_LH, x_HL, x_HH)
        return x_reconstructed


class CustomImageDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.image_paths = [os.path.join(root, fname) for fname in os.listdir(root) if fname.endswith(('jpg', 'png', 'jpeg'))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        path = self.image_paths[index]
        image = Image.open(path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, image

class L1Loss(nn.Module):
    def __init__(self):
        super(L1Loss, self).__init__()
        self.l1_loss = nn.L1Loss()

    def forward(self, predicted, target):
        return self.l1_loss(predicted, target)

class ColorConsistencyLoss(nn.Module):
    def __init__(self):
        super(ColorConsistencyLoss, self).__init__()

    def forward(self, x_GT, x_pred):
        x_GT_np = x_GT.cpu().numpy().transpose(0, 2, 3, 1)
        x_pred_np = x_pred.cpu().numpy().transpose(0, 2, 3, 1)

        x_GT_lab = np.array([cv2.cvtColor(img, cv2.COLOR_RGB2LAB) for img in x_GT_np])
        x_pred_lab = np.array([cv2.cvtColor(img, cv2.COLOR_RGB2LAB) for img in x_pred_np])

        x_GT_lab = torch.tensor(x_GT_lab).permute(0, 3, 1, 2).to(x_GT.device)
        x_pred_lab = torch.tensor(x_pred_lab).permute(0, 3, 1, 2).to(x_pred.device)

        color_loss = nn.L1Loss()(x_GT_lab, x_pred_lab)
        return color_loss

class TotalVariationLoss(nn.Module):
    def __init__(self):
        super(TotalVariationLoss, self).__init__()

    def forward(self, x):
        tv_h = torch.mean(torch.abs(x[:, :, 1:, :] - x[:, :, :-1, :]))
        tv_w = torch.mean(torch.abs(x[:, :, :, 1:] - x[:, :, :, :-1]))
        return tv_h + tv_w

def compute_total_loss(x_GT, x_HR_pred, x_L, x_B, lambda_res=4, lambda_tv=0.1, lambda_color=0.5):
    l1_loss_fn = L1Loss()
    color_loss_fn = ColorConsistencyLoss()
    tv_loss_fn = TotalVariationLoss()

    L_HR = l1_loss_fn(x_HR_pred, x_GT)
    L_LR = l1_loss_fn(x_L, x_B)

    L_TV = tv_loss_fn(x_HR_pred)

    L_color = color_loss_fn(x_GT, x_HR_pred)

    L_total = (lambda_res * L_HR) + (lambda_tv * L_TV) + (lambda_color * L_color) + (L_LR*0)

    return L_total, {'L_HR': L_HR.item(), 'L_LR': L_LR.item(), 'L_TV': L_TV.item(), 'L_color': L_color.item()}

def train_step(model, optimizer, data_loader, lambda_res=4, lambda_tv=0.1, lambda_color=0.5):
    model.train()
    total_loss_val = 0.0
    for i, (x, _) in enumerate(data_loader):
        x_L = model(x)
        x_HR_pred = model.inverse(x_L)
        x_B = F.interpolate(x, scale_factor=0.5, mode='bicubic', align_corners=True)

        total_loss, loss_details = compute_total_loss(x, x_HR_pred, x_L, x_B, lambda_res, lambda_tv, lambda_color)
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        total_loss_val += total_loss.item()

        print(f"Batch {i+1}/{len(data_loader)} - L_HR: {loss_details['L_HR']:.4f}, "
              f"L_LR: {loss_details['L_LR']:.4f}, L_TV: {loss_details['L_TV']:.4f}, "
              f"L_color: {loss_details['L_color']:.4f}")

    return total_loss_val / len(data_loader)

transform = transforms.Compose([
    transforms.RandomCrop((256, 256), padding=16, padding_mode='reflect'),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

# **DATASET LOADING** !

In [ ]:
# Define dataset and dataloader
dataset = CustomImageDataset(root='/content/data/DIV2K_train_HR/DIV2K_train_HR', transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# **MODEL INITIALIZATION**

In [ ]:
model = IIRModule(in_channels=3, num_blocks=4)


# Define optimizer and learning rate scheduler
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)

# **TRAINING LOOP** !

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    avg_loss = train_step(model, optimizer, dataloader, lambda_res=100, lambda_tv=0.1, lambda_color=0.5)
    print(f"Epoch {epoch+1}/{num_epochs}, Avg Loss: {avg_loss:.4f}")


Batch 1/50 - L_HR: 0.0340, L_LR: 0.4758, L_TV: 0.0491, L_color: 3.3880
Batch 2/50 - L_HR: 0.0255, L_LR: 0.5038, L_TV: 0.0422, L_color: 2.7205
Batch 3/50 - L_HR: 0.0364, L_LR: 0.5157, L_TV: 0.0542, L_color: 3.6868
Batch 4/50 - L_HR: 0.0380, L_LR: 0.4861, L_TV: 0.0540, L_color: 3.8761
Batch 5/50 - L_HR: 0.0267, L_LR: 0.5094, L_TV: 0.0443, L_color: 2.6157
Batch 6/50 - L_HR: 0.0344, L_LR: 0.5520, L_TV: 0.0515, L_color: 3.5417
Batch 7/50 - L_HR: 0.0261, L_LR: 0.4953, L_TV: 0.0444, L_color: 2.5821
Batch 8/50 - L_HR: 0.0300, L_LR: 0.4772, L_TV: 0.0507, L_color: 2.7906
Batch 9/50 - L_HR: 0.0222, L_LR: 0.5741, L_TV: 0.0344, L_color: 2.3896
Batch 10/50 - L_HR: 0.0270, L_LR: 0.5732, L_TV: 0.0388, L_color: 2.6996
Batch 11/50 - L_HR: 0.0289, L_LR: 0.5723, L_TV: 0.0509, L_color: 2.7924
Batch 12/50 - L_HR: 0.0280, L_LR: 0.5218, L_TV: 0.0453, L_color: 2.4845
Batch 13/50 - L_HR: 0.0290, L_LR: 0.5501, L_TV: 0.0453, L_color: 3.0073
Batch 14/50 - L_HR: 0.0361, L_LR: 0.6061, L_TV: 0.0584, L_color: 3.3617
B

# **DOWNLOADING TRAINED MODEL PTH FILE**

In [ ]:
!gdown 1jF9nVDTLIuCpdI6GYABqcIuMOgLpBRF0

Downloading...
From: https://drive.google.com/uc?id=1jF9nVDTLIuCpdI6GYABqcIuMOgLpBRF0
To: /content/iig_module_loss_4_8_epoch_10_checkpoint.pth
100% 88.1k/88.1k [00:00<00:00, 111MB/s]


# **LOADING TRAINED MODEL STATE DICT**

In [ ]:
state_dict = torch.load("/content/iig_module_loss_4_8_epoch_10_checkpoint.pth")
model.load_state_dict(state_dict['model_state_dict'])
optimizer.load_state_dict(state_dict['optimizer_state_dict'])
scheduler.load_state_dict(state_dict['scheduler_state_dict'])

<ipython-input-10-9c705737ed3d>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("/content/iig_module_loss_4_8_epoch_10_checkpoint.pth")


# **SETTING MODEL TO EVAL MODE**

In [ ]:
model.eval()

IIRModule(
  (dwt): DWTForward()
  (iwt): DWTInverse()
  (invertible_blocks): ModuleList(
    (0-3): 4 x InvertibleBlock(
      (phi): Sequential(
        (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
        (1): ReLU()
        (2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
      )
      (rho): Sequential(
        (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
        (1): ReLU()
        (2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
      )
      (eta): Sequential(
        (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
        (1): ReLU()
        (2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
      )
      (alpha): Sigmoid()
    )
  )
  (highpass_estimator): HighPassEstimator(
    (residual_block): Sequential(
      (0): Co

In [ ]:
for param in model.parameters():
    param.requires_grad = False

# **FUNCTION DEFINITION FOR PSNR**

In [ ]:

def calculate_psnr(original_image, reconstructed_image):
    mse = torch.mean((original_image - reconstructed_image) ** 2)
    if mse == 0:  # No noise is present in the signal
        return float('inf')  # Infinite PSNR
    psnr = 20 * torch.log10(1.0 / torch.sqrt(mse))  # Assuming the image is normalized to [0, 1]
    return psnr.item()


# **TESTING IMAGES WITH MODEL**

In [ ]:
def save_best_images(model, input_image, target_image, num_iterations=20, lambda_res=40, lambda_tv=0.1, lambda_color=0.1):
    l1_loss_fn = L1Loss()
    color_loss_fn = ColorConsistencyLoss()
    tv_loss_fn = TotalVariationLoss()

    min_loss = float('inf')
    best_compressed_image = None
    best_reconstructed_image = None

    for i in range(num_iterations):
        compressed_image = model(input_image)
        reconstructed_image = model.inverse(compressed_image, None)
        x_B = F.interpolate(input_image, scale_factor=0.5, mode='bicubic', align_corners=True)

        L_total, loss_details = compute_total_loss(input_image, reconstructed_image, compressed_image, x_B, lambda_res, lambda_tv, lambda_color)

        if L_total.item() < min_loss:
            min_loss = L_total.item()
            best_compressed_image = compressed_image.clone()
            best_reconstructed_image = reconstructed_image.clone()

        print(f"Iteration {i+1}/{num_iterations}: Loss = {L_total.item()}")

    compressed_image_pil = transforms.ToPILImage()(best_compressed_image.squeeze(0).cpu())
    compressed_image_pil.save('best_compressed_img.jpg')

    reconstructed_image_pil = transforms.ToPILImage()(best_reconstructed_image.squeeze(0).cpu())
    reconstructed_image_pil.save('best_reconstructed_img.jpg')

    input_image_pil = transforms.ToPILImage()(input_image.squeeze(0).cpu())
    input_image_pil.save('input_img.jpg')
    print(f"Minimum Loss: {min_loss}")

    # Calculate PSNR
    psnr = calculate_psnr(target_image, best_reconstructed_image)
    print(f"PSNR: {psnr:.2f} dB")

    original_image_size = input_image.numel() * input_image.element_size()
    compressed_image_size = best_compressed_image.numel() * best_compressed_image.element_size()
    compression_ratio = original_image_size / compressed_image_size
    print(f"Compression Ratio: {compression_ratio:.2f}")

# The Path of the Input Image to be tested
image = Image.open('/content/data/DIV2K_valid_HR/DIV2K_valid_HR/0830.png')

transform = transforms.Compose([transforms.ToTensor()])
input_image = transform(image).unsqueeze(0)
target_image = transform(image).unsqueeze(0)

save_best_images(model, input_image, target_image, num_iterations=10)


Iteration 1/10: Loss = 1.7657116651535034
Iteration 2/10: Loss = 1.7949259281158447
Iteration 3/10: Loss = 1.8604978322982788
Iteration 4/10: Loss = 1.8470524549484253
Iteration 5/10: Loss = 2.0462262630462646
Iteration 6/10: Loss = 1.6847890615463257
Iteration 7/10: Loss = 1.7069395780563354
Iteration 8/10: Loss = 1.7736010551452637
Iteration 9/10: Loss = 1.574975848197937
Iteration 10/10: Loss = 1.795569896697998
Minimum Loss: 1.574975848197937
PSNR: 25.71 dB
Compression Ratio: 4.00


# **CALCULATING BPP VALUE OF IMAGE**

In [ ]:
def calculate_bpp(image_path):
    with Image.open(image_path) as img:
        width, height = img.size
        num_channels = len(img.getbands())

    file_size_bytes = os.path.getsize(image_path)

    total_bits = file_size_bytes * 8
    total_pixels = width * height * num_channels
    bpp = total_bits / total_pixels

    return bpp, width, height, num_channels, file_size_bytes

image_path = 'best_compressed_img.jpg'
bpp_value, width, height, num_channels, file_size_bytes = calculate_bpp(image_path)

print(f"Width: {width}, Height: {height}, Channels: {num_channels}, File Size: {file_size_bytes} bytes")
print(f"Bits per Pixel (bpp): {bpp_value:.4f}")

Width: 1020, Height: 408, Channels: 3, File Size: 148311 bytes
Bits per Pixel (bpp): 0.9503


# **GIC MODULE DEFINITION**

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import pywt  # For DWT and IWT
import os
from torch.utils.data import DataLoader
from torchvision import datasets
import torch.optim as optim
from PIL import Image
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
from glob import glob

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1,padding_mode='reflect')
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, padding_mode='reflect')
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Skip connection layer if in_channels != out_channels
        self.skip_connection = None
        if in_channels != out_channels:
            self.skip_connection = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, padding=0, padding_mode='reflect')

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.skip_connection is not None:
            identity = self.skip_connection(identity)

        out += identity
        out = self.relu(out)
        return out

# Define the Attention Block
class AttentionBlock(nn.Module):
    def __init__(self, in_channels):
        super(AttentionBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        attn = self.conv(x)
        attn = self.sigmoid(attn)
        return x * attn

# Define the Importance Map Subnet
class ImportanceMapSubnet(nn.Module):
    def __init__(self, in_channels):
        super(ImportanceMapSubnet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 128, kernel_size=3, stride=1, padding=1, padding_mode='reflect')
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(128, 1, kernel_size=3, stride=1, padding=1, padding_mode='reflect')
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        return self.sigmoid(x)

# Define the GIC Module
class GICModule(nn.Module):
    def __init__(self):
        super(GICModule, self).__init__()
        # Initial Convolution to adjust input channels
        self.initial_conv = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, padding_mode='reflect')

        # Initial Residual Blocks and Attention Block
        self.resblocks = nn.Sequential(
            ResidualBlock(64, 64),
            ResidualBlock(64, 64),
            ResidualBlock(64, 64)
        )
        self.attention = AttentionBlock(64)

        # Branch 1 for importance map
        self.importance_branch = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, padding_mode='reflect'),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, padding_mode='reflect')
        )
        self.importance_map_subnet = ImportanceMapSubnet(128)

        # Branch 2 for feature extraction
        self.feature_branch = nn.Sequential(
            ResidualBlock(64, 64),
            ResidualBlock(64, 64),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, padding_mode='reflect'),
            AttentionBlock(128)
        )

        # Final layers for entropy encoding
        self.final_layers = nn.Sequential(
            ResidualBlock(128, 128),
            ResidualBlock(128, 128),
            nn.Conv2d(128, 3, kernel_size=3, stride=1, padding=1, padding_mode='reflect')
        )

    def forward(self, x):
        # Initial layers
        x = self.initial_conv(x)
        x = self.resblocks(x)
        a_b_o = self.attention(x)

        # Branch 1
        x1 = self.importance_branch(a_b_o)
        mask = self.importance_map_subnet(x1)

        # Branch 2
        x2 = self.feature_branch(a_b_o)

        # Masked feature
        masked_feature = x2 * mask

        # Final layers
        x = self.final_layers(masked_feature)

        return x

# **TRAINING LOOP !**

In [ ]:

def compute_bit_rate(xG_hat):
    num_bits_per_pixel = 8  # Adjust based on quantization
    batch_size, channels, height, width = xG_hat.shape  # Get the shape of the input tensor

    # Calculate the total number of pixels
    num_pixels = height * width * channels  # Total pixels in the batch

    # Normalize or clip the output to a desired range before calculating bitrate
    xG_hat_clipped = torch.clamp(xG_hat, 0, 1)  # Ensure the output is in the range [0, 1]

    # Calculate effective bitrate based on the quantization level
    effective_bitrate = (num_bits_per_pixel * num_pixels) / (batch_size * 1000.0)  # Return in kilobits
    return effective_bitrate

def compute_bpp(xG_hat, format='JPEG'):

    output_folder = "output_images"
    os.makedirs(output_folder, exist_ok=True)

    bpp_values = []

    # Assuming the tensor values are in range [0, 1], scale to [0, 255] and convert to 8-bit
    xG_hat = torch.clamp(xG_hat, 0, 1)  # Ensure values are within [0, 1]
    xG_hat = (xG_hat * 255).byte()  # Convert to [0, 255] and 8-bit representation

    # Get the shape of the tensor
    batch_size, channels, height, width = xG_hat.shape

    # Iterate over each image in the batch
    for i in range(batch_size):
        # Extract the ith image from the batch
        image_tensor = xG_hat[i]

        # Convert tensor to PIL image
        image = transforms.ToPILImage()(image_tensor.cpu())

        # Save the image to disk
        file_path = os.path.join(output_folder, f"image_{i}.{format.lower()}")
        image.save(file_path, format=format)

        # Get the size of the saved file in bytes
        file_size_bytes = os.path.getsize(file_path)

        # Convert file size to bits
        file_size_bits = file_size_bytes * 8

        # Calculate the total number of pixels (height * width)
        total_pixels = height * width

        # Calculate bits per pixel (bpp) for the current image
        bpp = file_size_bits / total_pixels

        # Store the bpp value
        bpp_values.append(bpp)

    # Calculate the average bpp across all images in the batch
    avg_bpp = sum(bpp_values)

    return avg_bpp





# Define the training function
def compression_loss(xH,xG, xG_hat):
    # L1 loss for distortion
    DG = nn.L1Loss()(xG_hat, xG)
    reconstructed_image = model.inverse(xG_hat)
    reconstruction_loss = nn.L1Loss()(reconstructed_image, xH)
    # Bit-rate loss placeholder (assuming RG is given or computed elsewhere)
    RG = compute_bpp(xG_hat)  # Placeholder
    lambda_C = 1  # Example weight, should be adjusted based on actual use case
    return reconstruction_loss*10 + DG + lambda_C * RG,DG,RG,reconstruction_loss

def train_gic_module(iig_model, gic_model, train_loader, gic_optimizer, num_epochs, device):
    gic_model.to(device)
    gic_model.train()

    for epoch in range(num_epochs):
        total_loss = 0.0
        for i, xH in enumerate(train_loader):
            xH = xH.to(device)

            # Pass original images through IIG to get compressed outputs
            xG = model(xH)  # Assuming IIG model is already defined and trained
            gic_optimizer.zero_grad()

            # Get the output from GIC Module
            xG_hat = gic_model(xG)
            loss, l1_loss, rg,reconstruction_loss = compression_loss(xH,xG, xG_hat)  # Unpack the losses

            loss.backward()

            # Uncomment to apply gradient clipping
            torch.nn.utils.clip_grad_norm_(gic_model.parameters(), max_norm=1.0)
            gic_optimizer.step()

            total_loss += loss.item()
            print(f"Batch: {i+1}, Total Loss: {loss:.4f}, L1 Loss: {l1_loss:.4f}, Bitrate RG: {rg:.4f}, Reconstruction Loss: {reconstruction_loss:.4f}")

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")
        torch.save({
        'epoch': epoch,
        'model_state_dict': gic_model.state_dict(),
        'optimizer_state_dict': gic_optimizer.state_dict(),
        }, f"/content/drive/MyDrive/GIC_MODULE1/gic_module_epoch_{epoch+1}_checkpoint.pth")
    print("Training completed.")
    return gic_model

# Custom dataset class remains the same
class CustomImageDataset(Dataset):
    def __init__(self, root, transform=None):
        self.image_paths = glob(os.path.join(root, "*.png"))  # Adjust for your image format
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img

# Hyperparameters remain the same
batch_size = 4
num_epochs = 10
learning_rate = 1e-4

# Device configuration remains the same
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Transformations for the training dataset remain the same
transform = transforms.Compose([
    transforms.RandomCrop((256, 256), padding=16, padding_mode='reflect'),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

# Load datasets
train_dataset = CustomImageDataset(root='/content/data/DIV2K_train_HR/DIV2K_train_HR', transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# Initialize model and optimizer
gic_model = GICModule()
gic_optimizer = optim.Adam(gic_model.parameters(), lr=learning_rate)

# Train the GIC module using the output from the IIG module
trained_model = train_gic_module(model, gic_model, train_loader, gic_optimizer, num_epochs, device)

# Save the trained model
torch.save(trained_model.state_dict(), 'gic_module.pth')
print("Model saved as 'gic_module.pth'.")


Batch: 1, Total Loss: 16.1587, L1 Loss: 1.1515, Bitrate RG: 9.2456, Reconstruction Loss: 0.5762
Batch: 2, Total Loss: 17.3492, L1 Loss: 1.2847, Bitrate RG: 9.5835, Reconstruction Loss: 0.6481
Batch: 3, Total Loss: 19.4990, L1 Loss: 1.0696, Bitrate RG: 13.0659, Reconstruction Loss: 0.5363
Batch: 4, Total Loss: 15.5686, L1 Loss: 0.9295, Bitrate RG: 10.0103, Reconstruction Loss: 0.4629
Batch: 5, Total Loss: 18.0839, L1 Loss: 1.0689, Bitrate RG: 11.6567, Reconstruction Loss: 0.5358
Batch: 6, Total Loss: 17.3369, L1 Loss: 0.8773, Bitrate RG: 12.0063, Reconstruction Loss: 0.4453
Batch: 7, Total Loss: 15.3392, L1 Loss: 0.6736, Bitrate RG: 11.1797, Reconstruction Loss: 0.3486
Batch: 8, Total Loss: 13.1954, L1 Loss: 0.4879, Bitrate RG: 10.2144, Reconstruction Loss: 0.2493
Batch: 9, Total Loss: 13.1884, L1 Loss: 0.3673, Bitrate RG: 10.9741, Reconstruction Loss: 0.1847
Batch: 10, Total Loss: 16.9671, L1 Loss: 0.3486, Bitrate RG: 14.9199, Reconstruction Loss: 0.1699
Batch: 11, Total Loss: 15.2211,

# **SAVE THE TRAINED MODEL !**

In [ ]:
torch.save({
        'epoch': 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        }, f"/content/drive/MyDrive/GIC_MODULE/gic_module_epoch_1_checkpoint.pth")

# **MODEL INITIALIZATION**

In [ ]:
learning_rate = 1e-4
gic_model = GICModule()
gic_optimizer = optim.Adam(gic_model.parameters(), lr=learning_rate)

# **DOWNLOADING GIC MODULE PTH FILE**

In [ ]:
!gdown 1-5tgCIe_Qp4PLZDINLvLn1NQ4mjY4HlR

Downloading...
From: https://drive.google.com/uc?id=1-5tgCIe_Qp4PLZDINLvLn1NQ4mjY4HlR
To: /content/gic_module_epoch_8_checkpoint.pth
100% 17.3M/17.3M [00:00<00:00, 127MB/s] 


# **LOADING TRAINED MODEL**

In [ ]:
checkpoint = torch.load("/content/gic_module_epoch_8_checkpoint.pth")
gic_model.load_state_dict(checkpoint['model_state_dict'])
gic_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
start_epoch = checkpoint['epoch']

<ipython-input-20-e3d6e2b78f2d>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("/content/gic_module_epoch_8_checkpoint.pth")


# **SETTING GIC MODEL TO EVAL MODE**

In [ ]:
gic_model.eval()

GICModule(
  (initial_conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
  (resblocks): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
      (bn2): BatchNorm2d(64, eps=1e-05, moment

In [ ]:
for param in gic_model.parameters():
    param.requires_grad = False

# **TESTING IMAGES WITH MODEL**

In [ ]:
def save_best_images(model, input_image, target_image, num_iterations=20, lambda_res=40, lambda_tv=0.1, lambda_color=0.1):
    l1_loss_fn = L1Loss()
    color_loss_fn = ColorConsistencyLoss()
    tv_loss_fn = TotalVariationLoss()

    min_loss = float('inf')
    best_compressed_image = None
    best_reconstructed_image = None

    for i in range(num_iterations):
        generated_image = model(input_image)
        compressed_image = gic_model(generated_image)
        reconstructed_image = model.inverse(compressed_image, None)
        x_B = F.interpolate(input_image, scale_factor=0.5, mode='bicubic', align_corners=True)

        L_total, loss_details = compute_total_loss(input_image, reconstructed_image, compressed_image, x_B, lambda_res, lambda_tv, lambda_color)

        if L_total.item() < min_loss:
            min_loss = L_total.item()
            best_compressed_image = compressed_image.clone()
            best_reconstructed_image = reconstructed_image.clone()
            best_generated_image = generated_image.clone()

        print(f"Iteration {i+1}/{num_iterations}: Loss = {L_total.item()}")

    generated_image_pil = transforms.ToPILImage()(best_generated_image.squeeze(0).cpu())
    generated_image_pil.save('best_generated_img.jpg')

    compressed_image_pil = transforms.ToPILImage()(best_compressed_image.squeeze(0).cpu())
    compressed_image_pil.save('best_compressed_img.jpg')

    reconstructed_image_pil = transforms.ToPILImage()(best_reconstructed_image.squeeze(0).cpu())
    reconstructed_image_pil.save('best_reconstructed_img.jpg')

    input_image_pil = transforms.ToPILImage()(input_image.squeeze(0).cpu())
    input_image_pil.save('input_img.jpg')
    print(f"Minimum Loss: {min_loss}")

    # Calculate PSNR
    psnr = calculate_psnr(target_image, best_reconstructed_image)
    print(f"PSNR: {psnr:.2f} dB")

    original_image_size = input_image.numel() * input_image.element_size()
    compressed_image_size = best_compressed_image.numel() * best_compressed_image.element_size()
    compression_ratio = original_image_size / compressed_image_size
    print(f"Compression Ratio: {compression_ratio:.2f}")

# The Path of the Input Image to be tested
image = Image.open('/content/data/DIV2K_valid_HR/DIV2K_valid_HR/0810.png')

transform = transforms.Compose([
    transforms.ToTensor()])
input_image = transform(image).unsqueeze(0)
target_image = transform(image).unsqueeze(0)

save_best_images(model, input_image, target_image, num_iterations=10)


Iteration 1/10: Loss = 1.979002833366394
Iteration 2/10: Loss = 2.0449225902557373
Iteration 3/10: Loss = 1.8207253217697144
Iteration 4/10: Loss = 1.9454832077026367
Iteration 5/10: Loss = 1.7434587478637695
Iteration 6/10: Loss = 1.9447134733200073
Iteration 7/10: Loss = 1.7524724006652832
Iteration 8/10: Loss = 1.7784042358398438
Iteration 9/10: Loss = 1.7501094341278076
Iteration 10/10: Loss = 1.7687532901763916
Minimum Loss: 1.7434587478637695
PSNR: 26.23 dB
Compression Ratio: 4.00


# **CALCULATING BPP VALUE OF IMAGE**

In [ ]:
def calculate_bpp(image_path):
    with Image.open(image_path) as img:
        width, height = img.size
        num_channels = len(img.getbands())

    file_size_bytes = os.path.getsize(image_path)

    total_bits = file_size_bytes * 8
    total_pixels = width * height * num_channels
    bpp = total_bits / total_pixels

    return bpp, width, height, num_channels, file_size_bytes

image_path = 'best_compressed_img.jpg'
bpp_value, width, height, num_channels, file_size_bytes = calculate_bpp(image_path)

print(f"Width: {width}, Height: {height}, Channels: {num_channels}, File Size: {file_size_bytes} bytes")
print(f"Bits per Pixel (bpp): {bpp_value:.4f}")

image_path = 'best_generated_img.jpg'
bpp_value, width, height, num_channels, file_size_bytes = calculate_bpp(image_path)

print(f"Width: {width}, Height: {height}, Channels: {num_channels}, File Size: {file_size_bytes} bytes")
print(f"Bits per Pixel (bpp): {bpp_value:.4f}")

image_path = 'input_img.jpg'
bpp_value, width, height, num_channels, file_size_bytes = calculate_bpp(image_path)

print(f"Width: {width}, Height: {height}, Channels: {num_channels}, File Size: {file_size_bytes} bytes")
print(f"Bits per Pixel (bpp): {bpp_value:.4f}")

Width: 1020, Height: 768, Channels: 3, File Size: 303148 bytes
Bits per Pixel (bpp): 1.0320
Width: 1020, Height: 768, Channels: 3, File Size: 310595 bytes
Bits per Pixel (bpp): 1.0573
Width: 2040, Height: 1536, Channels: 3, File Size: 482019 bytes
Bits per Pixel (bpp): 0.4102


# **CIA MODULE DEFINTION**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define SoftThresholding
class SoftThresholding(nn.Module):
    def __init__(self, lambda_val):
        super(SoftThresholding, self).__init__()
        self.lambda_val = lambda_val

    def forward(self, x):
        return torch.sign(x) * F.relu(torch.abs(x) - self.lambda_val)

# Define ConvBlock
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.relu(self.conv(x))

# Define LearnedConvolutionalSparseCoding
class LearnedConvolutionalSparseCoding(nn.Module):
    def __init__(self, channels, iterations=5):
        super(LearnedConvolutionalSparseCoding, self).__init__()
        self.iterations = iterations
        self.conv1 = ConvBlock(channels, channels)
        self.conv2 = ConvBlock(channels, channels)
        self.soft_threshold = SoftThresholding(lambda_val=0.1)

    def forward(self, x):
        u = self.conv1(x)
        for _ in range(self.iterations):
            u = self.soft_threshold(u - self.conv2(u) + self.conv1(x))
        return u

# Define ResidualChannelAttentionBlock
class ResidualChannelAttentionBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(ResidualChannelAttentionBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.global_pool = nn.AdaptiveAvgPool2d(1)

        reduced_channels = max(channels // reduction, 1)
        self.fc1 = nn.Conv2d(channels, reduced_channels, kernel_size=1)
        self.fc2 = nn.Conv2d(reduced_channels, channels, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        residual = x
        x = self.relu(self.conv1(x))
        x = self.conv2(x)
        attention = self.global_pool(x)
        attention = self.relu(self.fc1(attention))
        attention = self.sigmoid(self.fc2(attention))
        x = x * attention
        return x + residual

# Define CIA_Module
class CIA_Module(nn.Module):
    def __init__(self, channels=3):
        super(CIA_Module, self).__init__()
        self.lcsc = LearnedConvolutionalSparseCoding(channels)
        self.rcab1 = ResidualChannelAttentionBlock(channels)
        self.rcab2 = ResidualChannelAttentionBlock(channels)
        self.final_conv = nn.Conv2d(channels, channels, kernel_size=3, padding=1)

    def forward(self, x):
        x = self.lcsc(x)
        residual = x
        x = self.rcab1(x)
        x = self.rcab2(x)
        x = self.final_conv(x)
        x = x + residual
        return x

In [ ]:
def train_cia_module(cia_model, dataloader, device, num_epochs=20):
    cia_model.train()
    cia_optimizer = optim.Adam(cia_model.parameters(), lr=1e-4)
    l1_loss = nn.L1Loss()

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for batch_idx, (data, _) in enumerate(dataloader):
            data = data.to(device)
            x_G = data  # Assume data is the compressed image x̂_G

            # Forward pass
            generated_image = model(x_G)
            compressed_image = gic_model(generated_image)
            output = cia_model(compressed_image)

            # Enhancement loss: comparing enhanced output to original compressed input
            loss = l1_loss(output, generated_image)

            # Backward pass and optimization
            cia_optimizer.zero_grad()
            loss.backward()
            cia_optimizer.step()

            batch_loss = loss.item()
            epoch_loss += batch_loss

            # Print batch loss
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(dataloader)}], Batch Loss: {batch_loss:.4f}")

        # Print epoch loss
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(dataloader):.4f}")

        # Save model checkpoint
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': cia_model.state_dict(),
            'optimizer_state_dict': cia_optimizer.state_dict(),
        }, f"/content/drive/MyDrive/CIA_MODULE/cia_module_epoch_{epoch+1}_checkpoint.pth")

    # Freeze the model parameters after training
    for param in cia_model.parameters():
        param.requires_grad = False

    print("CIA Module training complete and model parameters are frozen.")
    print("Trained CIA module saved as 'cia_module_epoch_10_checkpoint.pth'.")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset with transformations including random crop and data augmentation
transform = transforms.Compose([
    transforms.RandomCrop((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

# Assuming images are located in the directory /content/data/
dataset = datasets.ImageFolder(root='/content/data/', transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Initialize the CIA module
cia_module = CIA_Module().to(device)

# Train the CIA module
train_cia_module(cia_module, dataloader, device)


Epoch [1/20], Batch [1/225], Batch Loss: 1.2009
Epoch [1/20], Batch [2/225], Batch Loss: 0.6583
Epoch [1/20], Batch [3/225], Batch Loss: 0.9770
Epoch [1/20], Batch [4/225], Batch Loss: 0.7848
Epoch [1/20], Batch [5/225], Batch Loss: 0.9976
Epoch [1/20], Batch [6/225], Batch Loss: 0.9230
Epoch [1/20], Batch [7/225], Batch Loss: 0.9297
Epoch [1/20], Batch [8/225], Batch Loss: 0.8323
Epoch [1/20], Batch [9/225], Batch Loss: 0.6832
Epoch [1/20], Batch [10/225], Batch Loss: 1.1313
Epoch [1/20], Batch [11/225], Batch Loss: 0.9137
Epoch [1/20], Batch [12/225], Batch Loss: 1.0179
Epoch [1/20], Batch [13/225], Batch Loss: 0.8708
Epoch [1/20], Batch [14/225], Batch Loss: 1.1125
Epoch [1/20], Batch [15/225], Batch Loss: 0.7734
Epoch [1/20], Batch [16/225], Batch Loss: 1.1695
Epoch [1/20], Batch [17/225], Batch Loss: 0.7713
Epoch [1/20], Batch [18/225], Batch Loss: 0.6848
Epoch [1/20], Batch [19/225], Batch Loss: 0.8073
Epoch [1/20], Batch [20/225], Batch Loss: 1.1164
Epoch [1/20], Batch [21/225],

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define Learnable Soft Thresholding
class SoftThresholding(nn.Module):
    def __init__(self, initial_lambda_val=0.1):
        super(SoftThresholding, self).__init__()
        self.lambda_val = nn.Parameter(torch.tensor(initial_lambda_val))  # Learnable parameter

    def forward(self, x):
        return torch.sign(x) * F.relu(torch.abs(x) - self.lambda_val)

# Define ConvBlock
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.relu(self.conv(x))

# Multi-Scale Aggregation Block
class MultiScaleConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(MultiScaleConvBlock, self).__init__()
        self.conv1x1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0)
        self.conv3x3 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv5x5 = nn.Conv2d(in_channels, out_channels, kernel_size=5, padding=2)
        self.conv7x7 = nn.Conv2d(in_channels, out_channels, kernel_size=7, padding=3)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        out1 = self.conv1x1(x)
        out2 = self.conv3x3(x)
        out3 = self.conv5x5(x)
        out4 = self.conv7x7(x)
        out = out1 + out2 + out3 + out4
        return self.relu(out)

# Learned Convolutional Sparse Coding
class LearnedConvolutionalSparseCoding(nn.Module):
    def __init__(self, channels, iterations=5):
        super(LearnedConvolutionalSparseCoding, self).__init__()
        self.iterations = iterations
        self.conv1 = ConvBlock(channels, channels)
        self.conv2 = ConvBlock(channels, channels)
        self.soft_threshold = SoftThresholding(initial_lambda_val=0.1)

    def forward(self, x):
        u = self.conv1(x)
        for _ in range(self.iterations):
            u = self.soft_threshold(u - self.conv2(u) + self.conv1(x))
        return u

# Residual Channel Attention Block with Spatial Attention
class ResidualChannelAttentionBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(ResidualChannelAttentionBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        reduced_channels = max(channels // reduction, 1)
        self.fc1 = nn.Conv2d(channels, reduced_channels, kernel_size=1)
        self.fc2 = nn.Conv2d(reduced_channels, channels, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

        # Spatial Attention
        self.spatial_conv = nn.Conv2d(2, 1, kernel_size=7, padding=3)
        self.spatial_sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Channel Attention
        residual = x
        x = self.relu(self.conv1(x))
        x = self.conv2(x)
        attention = self.global_pool(x)
        attention = self.relu(self.fc1(attention))
        attention = self.sigmoid(self.fc2(attention))
        x = x * attention

        # Spatial Attention
        max_pool = torch.max(x, dim=1, keepdim=True)[0]
        avg_pool = torch.mean(x, dim=1, keepdim=True)
        spatial_attention = self.spatial_sigmoid(self.spatial_conv(torch.cat([max_pool, avg_pool], dim=1)))
        x = x * spatial_attention

        return x + residual

# Define CIA_Module with deeper RCAB and Multi-Scale Block
class CIA_Module(nn.Module):
    def __init__(self, channels=3, rcab_depth=3):
        super(CIA_Module, self).__init__()
        self.lcsc = LearnedConvolutionalSparseCoding(channels)

        # Deblocking and Multi-Scale Aggregation
        self.multi_scale_block = MultiScaleConvBlock(channels, channels)

        # Enhancement Group with deeper RCAB
        self.rcabs = nn.Sequential(
            *[ResidualChannelAttentionBlock(channels) for _ in range(rcab_depth)]
        )
        self.final_conv = nn.Conv2d(channels, channels, kernel_size=3, padding=1)

    def forward(self, x):
        # Deblocking Group
        x = self.lcsc(x)
        x = self.multi_scale_block(x)
        residual = x

        # Enhancement Group
        x = self.rcabs(x)
        x = self.final_conv(x)
        return x + residual


In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

def train_cia_module(cia_model, dataloader, device, num_epochs=50):
    cia_model.train()
    cia_optimizer = optim.Adam(cia_model.parameters(), lr=1e-4)
    l1_loss = nn.L1Loss()

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for batch_idx, (data, _) in enumerate(dataloader):
            data = data.to(device)
            x_G = data  # Assume data is the compressed image x̂_G

            # Forward pass
            generated_image = model(x_G)               # Generate enhanced image
            compressed_image = gic_model(generated_image)  # Compress the enhanced image
            output = cia_model(compressed_image)       # Apply CIA module for enhancement
            reconstructed_image = model.inverse(output, None)  # Reconstruct original image

            # Calculate losses
            enhancement_loss = l1_loss(output, generated_image)  # Enhancement loss
            reconstruction_loss = l1_loss(x_G, reconstructed_image)  # Reconstruction loss
            total_loss = enhancement_loss*10 + reconstruction_loss*10       # Combined loss

            # Backward pass and optimization
            cia_optimizer.zero_grad()
            total_loss.backward()
            cia_optimizer.step()

            # Accumulate epoch loss and print batch losses
            batch_loss = total_loss.item()
            epoch_loss += batch_loss
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(dataloader)}], "
                  f"Enhancement Loss: {enhancement_loss.item():.4f}, "
                  f"Reconstruction Loss: {reconstruction_loss.item():.4f}, "
                  f"Total Loss: {batch_loss:.4f}")

        # Print average epoch loss
        print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {epoch_loss/len(dataloader):.4f}")

        # Save model checkpoint
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': cia_model.state_dict(),
            'optimizer_state_dict': cia_optimizer.state_dict(),
        }, f"/content/drive/MyDrive/CIA_MODULE2/cia_module_modified_epoch_{epoch+1}_checkpoint.pth")

    # Freeze the model parameters after training
    for param in cia_model.parameters():
        param.requires_grad = False

    print("CIA Module training complete and model parameters are frozen.")
    print("Trained CIA module saved as 'cia_module_epoch_{num_epochs}_checkpoint.pth'.")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset with transformations including random crop and data augmentation
transform = transforms.Compose([
    transforms.RandomCrop((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

# Assuming images are located in the directory /content/data/
dataset = datasets.ImageFolder(root='/content/data/', transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Initialize the CIA module
cia_module = CIA_Module().to(device)

# Train the CIA module
train_cia_module(cia_module, dataloader, device)


Streaming output truncated to the last 5000 lines.
Epoch [6/50], Batch [45/225], Enhancement Loss: 0.1697, Reconstruction Loss: 0.0936, Total Loss: 2.6332
Epoch [6/50], Batch [46/225], Enhancement Loss: 0.1272, Reconstruction Loss: 0.0739, Total Loss: 2.0117
Epoch [6/50], Batch [47/225], Enhancement Loss: 0.1030, Reconstruction Loss: 0.0603, Total Loss: 1.6325
Epoch [6/50], Batch [48/225], Enhancement Loss: 0.1449, Reconstruction Loss: 0.0786, Total Loss: 2.2354
Epoch [6/50], Batch [49/225], Enhancement Loss: 0.0994, Reconstruction Loss: 0.0565, Total Loss: 1.5590
Epoch [6/50], Batch [50/225], Enhancement Loss: 0.1696, Reconstruction Loss: 0.0895, Total Loss: 2.5908
Epoch [6/50], Batch [51/225], Enhancement Loss: 0.1233, Reconstruction Loss: 0.0675, Total Loss: 1.9081
Epoch [6/50], Batch [52/225], Enhancement Loss: 0.2566, Reconstruction Loss: 0.1399, Total Loss: 3.9656
Epoch [6/50], Batch [53/225], Enhancement Loss: 0.1943, Reconstruction Loss: 0.1027, Total Loss: 2.9704
Epoch [6/50],

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cia_model = CIA_Module().to(device)
cia_optimizer = optim.Adam(cia_model.parameters(), lr=1e-4)


# **DOWNLOADING TRAINED CIA MODEL**

In [ ]:
!gdown 12LKyN63RrvtiPaGAU2py6SnYfkc0Z_mA

Downloading...
From: https://drive.google.com/uc?id=12LKyN63RrvtiPaGAU2py6SnYfkc0Z_mA
To: /content/cia_module_modified_epoch_27_checkpoint.pth
100% 79.6k/79.6k [00:00<00:00, 69.9MB/s]


# **LOADING PTH FILE TO CIA MODEL**

In [ ]:
checkpoint = torch.load("/content/cia_module_modified_epoch_27_checkpoint.pth")
cia_model.load_state_dict(checkpoint['model_state_dict'])
cia_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
start_epoch = checkpoint['epoch']

# **UNFREEZING ALL MODULES FOR TRAINING**

In [ ]:
model.train()

IIRModule(
  (dwt): DWTForward()
  (iwt): DWTInverse()
  (invertible_blocks): ModuleList(
    (0-3): 4 x InvertibleBlock(
      (phi): Sequential(
        (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
        (1): ReLU()
        (2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
      )
      (rho): Sequential(
        (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
        (1): ReLU()
        (2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
      )
      (eta): Sequential(
        (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
        (1): ReLU()
        (2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
      )
      (alpha): Sigmoid()
    )
  )
  (highpass_estimator): HighPassEstimator(
    (residual_block): Sequential(
      (0): Co

In [ ]:
for param in model.parameters():
    param.requires_grad = True

In [ ]:
gic_model.train()

In [ ]:
for param in gic_model.parameters():
    param.requires_grad = True

In [ ]:
cia_model.train()

In [ ]:
for param in cia_model.parameters():
    param.requires_grad = True

# **TRAINING LOOP FOR WHOLE FRAMEWORK**

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np
import cv2
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define Color Consistency Loss
class ColorConsistencyLoss(nn.Module):
    def __init__(self):
        super(ColorConsistencyLoss, self).__init__()

    def forward(self, x_GT, x_pred):
        x_GT_np = x_GT.cpu().detach().numpy().transpose(0, 2, 3, 1)
        x_pred_np = x_pred.cpu().detach().numpy().transpose(0, 2, 3, 1)

        x_GT_lab = np.array([cv2.cvtColor(img, cv2.COLOR_RGB2LAB) for img in x_GT_np])
        x_pred_lab = np.array([cv2.cvtColor(img, cv2.COLOR_RGB2LAB) for img in x_pred_np])

        x_GT_lab = torch.tensor(x_GT_lab).permute(0, 3, 1, 2).to(x_GT.device)
        x_pred_lab = torch.tensor(x_pred_lab).permute(0, 3, 1, 2).to(x_pred.device)

        color_loss = nn.L1Loss()(x_GT_lab, x_pred_lab)
        return color_loss

# Load models and optimizers from checkpoints
checkpoint_gic = torch.load("/content/gic_module_epoch_8_checkpoint.pth")
gic_model.load_state_dict(checkpoint_gic['model_state_dict'])
gic_optimizer = optim.Adam(gic_model.parameters(), lr=1e-4)
gic_optimizer.load_state_dict(checkpoint_gic['optimizer_state_dict'])

checkpoint_iig = torch.load("/content/iig_module_loss_4_8_epoch_10_checkpoint.pth")
model.load_state_dict(checkpoint_iig['model_state_dict'])
optimizer = optim.Adam(model.parameters(), lr=1e-4)
optimizer.load_state_dict(checkpoint_iig['optimizer_state_dict'])
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
scheduler.load_state_dict(checkpoint_iig['scheduler_state_dict'])

checkpoint_cia = torch.load("/content/cia_module_modified_epoch_27_checkpoint.pth")
cia_model.load_state_dict(checkpoint_cia['model_state_dict'])
cia_optimizer = optim.Adam(cia_model.parameters(), lr=1e-4)
cia_optimizer.load_state_dict(checkpoint_cia['optimizer_state_dict'])

# Training function
def train_modules(model, gic_model, cia_model, dataloader, device, num_epochs=50):
    model.train()
    gic_model.train()
    cia_model.train()

    l1_loss = nn.L1Loss()
    color_consistency_loss = ColorConsistencyLoss()

    for epoch in range(num_epochs):
        epoch_loss = 0.0

        for batch_idx, (data, _) in enumerate(dataloader):
            data = data.to(device)
            x_G = data  # Compressed input image

            # Forward pass
            generated_image = model(x_G)                    # Enhanced image
            compressed_image = gic_model(generated_image)   # Compressed enhanced image
            output = cia_model(compressed_image)            # Enhanced compressed image
            reconstructed_image = model.inverse(output, None)  # Reconstructed original

            # Calculate losses
            reconstruction_loss = l1_loss(x_G, reconstructed_image)  # Reconstruction loss
            enhancement_loss = color_consistency_loss(x_G, reconstructed_image)  # Color consistency loss
            total_loss = reconstruction_loss*100 + enhancement_loss*0.5  # Combined total loss

            # Backward pass and optimization
            optimizer.zero_grad()
            gic_optimizer.zero_grad()
            cia_optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()
            gic_optimizer.step()
            cia_optimizer.step()

            # Accumulate epoch loss and print batch loss
            epoch_loss += total_loss.item()
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(dataloader)}], "
                  f"Reconstruction Loss: {reconstruction_loss.item():.4f}, "
                  f"Color Consistency Loss: {enhancement_loss.item():.4f}, "
                  f"Total Loss: {total_loss.item():.4f}")

        # Adjust learning rate
        scheduler.step()

        # Print average epoch loss
        print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {epoch_loss/len(dataloader):.4f}")

        # Save all modules in a single checkpoint file
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'gic_model_state_dict': gic_model.state_dict(),
            'cia_model_state_dict': cia_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'gic_optimizer_state_dict': gic_optimizer.state_dict(),
            'cia_optimizer_state_dict': cia_optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict()
        }, f"/content/drive/MyDrive/MODEL/all_models_epoch_{epoch+1}_checkpoint.pth")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset loading
transform = transforms.Compose([
    transforms.RandomCrop((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(root='/content/data/', transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Train all modules
train_modules(model, gic_model, cia_model, dataloader, device)


<ipython-input-23-1d01b6c2467e>:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_gic = torch.load("/content/gic_module_epoch_8_checkpoint.pth")
<ipython-input-23

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np
import cv2
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define Color Consistency Loss
class ColorConsistencyLoss(nn.Module):
    def __init__(self):
        super(ColorConsistencyLoss, self).__init__()

    def forward(self, x_GT, x_pred):
        x_GT_np = x_GT.cpu().detach().numpy().transpose(0, 2, 3, 1)
        x_pred_np = x_pred.cpu().detach().numpy().transpose(0, 2, 3, 1)

        x_GT_lab = np.array([cv2.cvtColor(img, cv2.COLOR_RGB2LAB) for img in x_GT_np])
        x_pred_lab = np.array([cv2.cvtColor(img, cv2.COLOR_RGB2LAB) for img in x_pred_np])

        x_GT_lab = torch.tensor(x_GT_lab, requires_grad=True).permute(0, 3, 1, 2).to(x_GT.device)
        x_pred_lab = torch.tensor(x_pred_lab, requires_grad=True).permute(0, 3, 1, 2).to(x_pred.device)

        color_loss = nn.L1Loss()(x_GT_lab, x_pred_lab)
        return color_loss

# Load models and optimizers from checkpoints
checkpoint_gic = torch.load("/content/gic_module_epoch_8_checkpoint.pth")
gic_model.load_state_dict(checkpoint_gic['model_state_dict'])
gic_optimizer = optim.Adam(gic_model.parameters(), lr=1e-4)
gic_optimizer.load_state_dict(checkpoint_gic['optimizer_state_dict'])

checkpoint_iig = torch.load("/content/iig_module_loss_4_8_epoch_10_checkpoint.pth")
model.load_state_dict(checkpoint_iig['model_state_dict'])
optimizer = optim.Adam(model.parameters(), lr=1e-4)
optimizer.load_state_dict(checkpoint_iig['optimizer_state_dict'])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
scheduler.load_state_dict(checkpoint_iig['scheduler_state_dict'])

checkpoint_cia = torch.load("/content/cia_module_modified_epoch_27_checkpoint.pth")
cia_model.load_state_dict(checkpoint_cia['model_state_dict'])
cia_optimizer = optim.Adam(cia_model.parameters(), lr=1e-4)
cia_optimizer.load_state_dict(checkpoint_cia['optimizer_state_dict'])

# Training function
def train_modules(model, gic_model, cia_model, dataloader, device, num_epochs=50):
    model.train()
    gic_model.train()
    cia_model.train()

    l1_loss = nn.L1Loss()
    color_consistency_loss = ColorConsistencyLoss()

    for epoch in range(num_epochs):
        epoch_loss = 0.0

        for batch_idx, (data, _) in enumerate(dataloader):
            data = data.to(device)
            data.requires_grad = True  # Ensure input data has requires_grad=True
            x_G = data  # Compressed input image

            # Forward pass
            generated_image = model(x_G)                    # Enhanced image
            compressed_image = gic_model(generated_image)   # Compressed enhanced image
            output = cia_model(compressed_image)            # Enhanced compressed image
            reconstructed_image = model.inverse(output, None)  # Reconstructed original
            reconstructed_image.requires_grad = True  # Ensure requires_grad=True

            # Calculate losses
            reconstruction_loss = l1_loss(x_G, reconstructed_image)  # Reconstruction loss
            enhancement_loss = color_consistency_loss(x_G, reconstructed_image)  # Color consistency loss
            total_loss = reconstruction_loss*100 + enhancement_loss  # Combined total loss

            # Backward pass and optimization
            optimizer.zero_grad()
            gic_optimizer.zero_grad()
            cia_optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()
            gic_optimizer.step()
            cia_optimizer.step()

            # Accumulate epoch loss and print batch loss
            epoch_loss += total_loss.item()
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(dataloader)}], "
                  f"Reconstruction Loss: {reconstruction_loss.item():.4f}, "
                  f"Color Consistency Loss: {enhancement_loss.item():.4f}, "
                  f"Total Loss: {total_loss.item():.4f}")

        # Adjust learning rate
        scheduler.step()

        # Print average epoch loss
        print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {epoch_loss/len(dataloader):.4f}")

        # Save all modules in a single checkpoint file
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'gic_model_state_dict': gic_model.state_dict(),
            'cia_model_state_dict': cia_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'gic_optimizer_state_dict': gic_optimizer.state_dict(),
            'cia_optimizer_state_dict': cia_optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict()
        }, f"/content/drive/MyDrive/MODEL/all_models_epoch_{epoch+1}_checkpoint.pth")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset loading
transform = transforms.Compose([
    transforms.RandomCrop((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(root='/content/data/', transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Train all modules
train_modules(model, gic_model, cia_model, dataloader, device)


<ipython-input-30-946a25011235>:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_gic = torch.load("/content/gic_module_epoch_8_checkpoint.pth")
<ipython-input-30

Streaming output truncated to the last 5000 lines.
Epoch [28/50], Batch [199/225], Reconstruction Loss: 0.0712, Color Consistency Loss: 9.7042, Total Loss: 16.8195
Epoch [28/50], Batch [200/225], Reconstruction Loss: 0.0772, Color Consistency Loss: 9.7410, Total Loss: 17.4602
Epoch [28/50], Batch [201/225], Reconstruction Loss: 0.0341, Color Consistency Loss: 4.2778, Total Loss: 7.6924
Epoch [28/50], Batch [202/225], Reconstruction Loss: 0.0577, Color Consistency Loss: 7.9295, Total Loss: 13.7003
Epoch [28/50], Batch [203/225], Reconstruction Loss: 0.0569, Color Consistency Loss: 6.1615, Total Loss: 11.8513
Epoch [28/50], Batch [204/225], Reconstruction Loss: 0.0468, Color Consistency Loss: 4.9949, Total Loss: 9.6710
Epoch [28/50], Batch [205/225], Reconstruction Loss: 0.0792, Color Consistency Loss: 7.7811, Total Loss: 15.6997
Epoch [28/50], Batch [206/225], Reconstruction Loss: 0.0527, Color Consistency Loss: 5.9320, Total Loss: 11.1980
Epoch [28/50], Batch [207/225], Reconstruction 

# **DOWNLOADING PTH FILE**

In [1]:
!gdown 11WOfLyfjil_kion2UoMzYnOTqrw1imp8

Downloading...
From: https://drive.google.com/uc?id=11WOfLyfjil_kion2UoMzYnOTqrw1imp8
To: /content/all_models_epoch_45_checkpoint.pth
100% 17.5M/17.5M [00:00<00:00, 160MB/s]


In [ ]:
# Load models and optimizers from checkpoints
checkpoint = torch.load("/content/all_models_epoch_45_checkpoint.pth")
gic_model.load_state_dict(checkpoint['model_state_dict'])
gic_optimizer = optim.Adam(gic_model.parameters(), lr=1e-4)
gic_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

model.load_state_dict(checkpoint['model_state_dict'])
optimizer = optim.Adam(model.parameters(), lr=1e-4)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

cia_model.load_state_dict(checkpoint['model_state_dict'])
cia_optimizer = optim.Adam(cia_model.parameters(), lr=1e-4)
cia_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
